# twitterapiv2

This library offers access through Scala to the twitter API v2 endpoints. Additional aids for pagination and throttle management are also provided.

### Set up

In [3]:
import $ivy.`com.typesafe.akka::akka-http-spray-json:10.2.4`
import $ivy.`com.typesafe.akka::akka-slf4j:2.6.8`
import $ivy.`com.typesafe.akka::akka-actor-typed:2.6.8`
import $ivy.`com.typesafe.akka::akka-stream:2.6.8`
import $ivy.`com.typesafe.akka::akka-http:10.2.4`

import $ivy.$                                               

import $ivy.$                                    

import $ivy.$                                          

import $ivy.$                                     

import $ivy.$                                    

Please, create the project `jar` by typing in the sbt prompt:

`sbt:twitterapiv2> package`

In [4]:
import $cp.target.`scala-2.13`.`twitterapiv2_2.13-0.1.jar`

import $cp.$                                              

### Imports and dependencies

In [5]:
import scala.concurrent.{Future, Await, ExecutionContext, duration}, duration._
import _root_.akka.actor.typed.ActorSystem
import _root_.akka.actor.typed.scaladsl.Behaviors
import scala.util.Success
import scala.util.Failure
import dev.habla.twitter.{v2_akka, v2}, v2_akka._

import scala.concurrent.{Future, Await, ExecutionContext, duration}, duration._

import _root_.akka.actor.typed.ActorSystem

import _root_.akka.actor.typed.scaladsl.Behaviors

import scala.util.Success

import scala.util.Failure

import dev.habla.twitter.{v2_akka, v2}, v2_akka._

Common dependencies for actor-based systems and Akka stream:

In [6]:
implicit val system = ActorSystem(Behaviors.empty, "TwitterV2")
implicit val ec = system.executionContext

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


system: ActorSystem[Any] = akka://TwitterV2
ec: concurrent.ExecutionContextExecutor = Dispatcher[akka.actor.default-dispatcher]

Obtain the bearer token from the environment. Remember to pass this variable to docker, in case you started this notebook from there (`$ docker run -e <var_name> ...`)

In [7]:
def bearerToken = scala.util.Properties.envOrElse("BEARER_TOKEN", "undefined")

defined function bearerToken

### Single requests

We can access programmatically different endpoints to obtain the response of a single request. For instance, we can create a [search/recent](https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent) request and obtain its response as follows:

In [ ]:

val response: Future[v2.recents.SingleResponse] = 
    v2.recents.SingleRequest("scala3", bearerToken, max_results=Some(10))
        .single

In [ ]:
response.map{ case v2.recents.Tweets(_, r, s) => (r,s) }

In [ ]:
show(Await.result(response, 1.second))

Similarly, we can [lookup a tweet](https://developer.twitter.com/en/docs/twitter-api/tweets/lookup/api-reference/get-tweets-id) by its identifier as follows:

In [ ]:
v2.lookupt.Request("787969995917656064", bearerToken)
        .single

### Pagination

In [8]:
import akka.stream.scaladsl._

import akka.stream.scaladsl._

But the library can also do pagination and throttle management for us automatically:

In [9]:
val response: Source[v2.recents.SingleResponse, akka.NotUsed] = 
    v2.recents.SingleRequest("scala3 -is:retweet", bearerToken)
        .stream

response: Source[v2.recents.SingleResponse, akka.NotUsed] = Source(SourceShape(flatten.out(4893099)))

In [13]:
response.take(2)
    .toMat(Sink.seq)(Keep.right) // RunnableGraph
    .run                         // Future

res12: Future[Seq[v2.recents.SingleResponse]] = Success(
  Vector(
    Tweets(
      Body(
        Some(
          List(
            Tweet(
              "1397895129571024896",
              "scala3\u306e\u30a4\u30f3\u30c7\u30f3\u30c8\u69cb\u6587\u3001\u305f\u3057\u304b\u306b\u3056\u308f\u3064\u304f\u3051\u3069\u3001\u3053\u308c\u3082\u30b9\u30b1\u30fc\u30e9\u30d6\u30eb\u3055\u3060\u3063\u305f\u308a\u3059\u308b\u3093\u304b\u306a\u3001\u3068\u304b\u601d\u3063\u305f\u3002\u3066\u304b\u3001\u30d6\u30ed\u30c3\u30af\u3068\u30a4\u30f3\u30c7\u30f3\u30c8\u3063\u3066\u540c\u5c45\u3067\u304d\u308b\u3093\u3060\u30fc\u3063\u3066\u3044\u3046\u306e\u3001\u9762\u767d\u3044\u3002\u305d\u306e\u30a4\u30f3\u30c7\u30f3\u30c8\u69cb\u6587\u306e\u62e1\u5f35\u3067ruby\u3063\u307d\u304f\u3082\u66f8\u3051\u308b\u306e\u3082\u9762\u767d\u3044\u3002",
              None,
              Some(JsString("139978757")),
              None,
              Some(JsString("1397895129571024896")),
              Some(JsString("2021-05-27T12:39:05.000Z")),
              None,
              None,
              None,
              Some(JsString("ja")),
              None,
              None,
              None,
              None,
              Some(
                JsObject(
                  TreeMap(
                    "like_count" -> JsNumber(1),
                    "quote_count" -> JsNumber(0),
                    "reply_count" -> JsNumber(0),
                    "retweet_count" -> JsNumber(0)
                  )
                )
...

Using this stream, we may take the last `N` tweets:

In [ ]:
def takeNTweets(request: v2.recents.SingleRequest)(n: Int): Future[Seq[v2.Tweet]] = 
    request.stream
        .mapConcat{ 
            case v2.recents.Tweets(v2.recents.Tweets.Body(Some(tweets), _, meta), _, _) => 
                tweets 
            case _ => List()
        }.take(n)
        .toMat(Sink.seq)(Keep.right)
        .run


In [ ]:
takeNTweets(v2.recents.SingleRequest("scala3", bearerToken, tweet_fields = Some("id,text,created_at")))(1)